In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [ ]:
!pip install keras_cv -q

In [ ]:
!pip install --upgrade keras

In [ ]:
!pip install segmentation_models

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import keras
from tensorflow.keras import ops
import keras_cv
import numpy as np
import segmentation_models as sm
from keras_cv.datasets.pascal_voc.segmentation import load as load_voc
from tensorflow.keras import backend as K

In [ ]:
train_ds = load_voc(split="sbd_train")
eval_ds = load_voc(split="sbd_eval")

In [ ]:
def preprocess_tfds_inputs(inputs):
    def unpackage_tfds_inputs(tfds_inputs):
        return {
            "images": tfds_inputs["image"],
            "segmentation_masks": tfds_inputs["class_segmentation"],
        }

    outputs = inputs.map(unpackage_tfds_inputs)
    outputs = outputs.map(keras_cv.layers.Resizing(height=224, width=224))
    outputs = outputs.batch(32, drop_remainder=True)
    return outputs


train_ds = preprocess_tfds_inputs(train_ds)
batch = train_ds.take(1).get_single_element()
keras_cv.visualization.plot_segmentation_mask_gallery(
    batch["images"],
    value_range=(0, 255),
    num_classes=21,  
    y_true=batch["segmentation_masks"],
    scale=3,
    rows=2,
    cols=2,
)

In [ ]:
eval_ds = preprocess_tfds_inputs(eval_ds)

In [ ]:
train_ds = train_ds.map(keras_cv.layers.RandomFlip())
train_ds = train_ds.map(keras_cv.layers.RandomRotation(factor=.1,segmentation_classes=21))

batch = train_ds.take(1).get_single_element()

keras_cv.visualization.plot_segmentation_mask_gallery(
    batch["images"],
    value_range=(0, 255),
    num_classes=21,
    y_true=batch["segmentation_masks"],
    scale=3,
    rows=2,
    cols=2,
)

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Dropout, concatenate
from tensorflow.keras.models import Model
def unet_model(input_size=(224,224, 3)):
    #build the model
    ####################
    #your code goes here
    ####################
    model = Model(inputs=[inputs], outputs=[outputs])
    return model


model = unet_model()


In [ ]:
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [ ]:
def jaccard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

In [ ]:
metrics=['accuracy', jaccard_coef]
#compile the model
####################
#your code goes here
####################

In [ ]:
model.summary()

In [ ]:
def dict_to_tuple(x):


    return x["images"], tf.one_hot(
        tf.cast(tf.squeeze(x["segmentation_masks"], axis=-1), "int32"), 21
    )


train_ds = train_ds.map(dict_to_tuple)

eval_ds = eval_ds.map(dict_to_tuple)



In [ ]:
model.fit(train_ds, validation_data=eval_ds, epochs=10 )

In [ ]:
activation='softmax'

LR = 0.001
optim = keras.optimizers.Adam(LR)


dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)


In [ ]:
BACKBONE1 = 'mobilenetv2'

n_classes=21
# define model
model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation)
model1.compile(optim, total_loss, metrics=metrics)
print(model1.summary())

In [ ]:
flag = True
for l in model1.layers:
  if  l.name=='decoder_stage0_upsampling':
    flag = ?
  if flag:
    l.trainable = ?

In [ ]:
model1.summary()

In [ ]:
model1.fit(train_ds, validation_data=eval_ds, epochs=10, batch_size=32 )

In [ ]:
#let all the layers be trained
####################
#your code goes here
####################

In [ ]:
LR = 0.000005
optim = keras.optimizers.Adam(LR)

In [ ]:
model1.compile(optim, total_loss, metrics=metrics)

In [ ]:
model1.fit(train_ds, validation_data=eval_ds, epochs=5, batch_size=32 )